In [1]:
import ray
ray.init(address='auto')
from pprint import pprint
pprint(ray.nodes())

2021-12-05 20:01:23,427	INFO worker.py:842 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379


[{'Alive': True,
  'MetricsExportPort': 62807,
  'NodeID': 'e26fe98a02765160d541d61f013d630d4235438b4ad23ee2b146dbd2',
  'NodeManagerAddress': '127.0.0.1',
  'NodeManagerHostname': 'LAPTOP-5F3V9FNM',
  'NodeManagerPort': 61450,
  'ObjectManagerPort': 61448,
  'ObjectStoreSocketName': 'tcp://127.0.0.1:64348',
  'RayletSocketName': 'tcp://127.0.0.1:64500',
  'Resources': {'CPU': 4.0,
                'GPU': 1.0,
                'memory': 1592052942.0,
                'node:127.0.0.1': 1.0,
                'object_store_memory': 796026470.0},
  'alive': True}]


In [4]:
import time

import ray
from ray.util.metrics import Counter, Gauge, Histogram

ray.shutdown()
ray.init(_metrics_export_port=8080)

@ray.remote
class MyActor:
    def __init__(self, name):
        self._curr_count = 0

        self.counter = Counter(
            "num_requests",
            description="Number of requests processed by the actor.",
            tag_keys=("actor_name", ))
        self.counter.set_default_tags({"actor_name": name})

        self.gauge = Gauge(
            "curr_count",
            description="Current count held by the actor. Goes up and down.",
            tag_keys=("actor_name", ))
        self.gauge.set_default_tags({"actor_name": name})

        self.histogram = Histogram(
            "request_latency",
            description="Latencies of requests in ms.",
            boundaries=[0.1, 1],
            tag_keys=("actor_name", ))
        self.histogram.set_default_tags({"actor_name": name})

    def process_request(self, num):
        start = time.time()
        self._curr_count += num

        # Increment the total request count.
        self.counter.inc()
        # Update the gauge to the new value.
        self.gauge.set(self._curr_count)
        # Record the latency for this request in ms.
        self.histogram.observe(1000 * (time.time() - start))

        return self._curr_count


print("Starting actor.")
my_actor = MyActor.remote("my_actor")
print("Calling actor.")
my_actor.process_request.remote(-10)
print("Calling actor.")
my_actor.process_request.remote(5)
print("Metrics should be exported.")
print("See http://localhost:8080 (this may take a few seconds to load).")

# Sleep so we can look at the metrics before exiting.
time.sleep(30)
print("Exiting!")

2021-12-05 20:02:32,408	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


Starting actor.
Calling actor.
Calling actor.
Metrics should be exported.
See http://localhost:8080 (this may take a few seconds to load).


2021-12-05 20:02:53,803	WARNING worker.py:1245 -- The actor or task with ID ffffffffffffffffd7edaf1ddecbbf7e698644fb01000000 cannot be scheduled right now. You can ignore this message if this Ray cluster is expected to auto-scale or if you specified a runtime_env for this actor or task, which may take time to install.  Otherwise, this is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increasing the resources available to this Ray cluster.
Required resources for this actor or task: {GPU: 1.000000}, {CPU: 1.000000}
Available resources on this node: {3.000000/4.000000 CPU, 48934800.000000 GiB/48934800.000000 GiB memory, 0.000000/1.000000 GPU, 24467400.000000 GiB/24467400.000000 GiB object_store_memory, 1.000000/1.000000 node:127.0.0.1}
 In total there are 0 pending tasks and 1 pending actors on this node.


(scheduler +1m42s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0, 'GPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
Exiting!
(scheduler +1m50s) Warning: The following resource request cannot be scheduled right now: {'GPU': 1.0, 'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


Debugger

In [ ]:
import ray

ray.shutdown()
ray.init()

@ray.remote
def f(x):
    breakpoint()
    return x * x

futures = [f.remote(i) for i in range(2)]
print(ray.get(futures))

2021-12-05 20:03:31,482	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8266


In [2]:
import time

from sklearn.datasets import load_iris
from sklearn.ensemble import GradientBoostingClassifier

import ray
from ray import serve

serve.start()

# Train model
iris_dataset = load_iris()
model = GradientBoostingClassifier()
model.fit(iris_dataset["data"], iris_dataset["target"])

# Define Ray Serve model,
@serve.deployment(route_prefix="/iris")
class BoostingModel:
    def __init__(self):
        self.model = model
        self.label_list = iris_dataset["target_names"].tolist()
        
    await def __call__(self, starlette_request):
        payload = await starlette_request.json()["vector"]
        print(f"Worker: received request with data: {payload}")

        prediction = self.model.predict([payload])[0]
        human_name = self.label_list[prediction]
        return {"result": human_name}

# Deploy model
serve.start()
BoostingModel.deploy()

time.sleep(3600.0)

SyntaxError: invalid syntax (Temp/ipykernel_8952/1572540588.py, line 23)

Logger

In [3]:
import ray
# Initiate a driver.
ray.init()

@ray.remote
def task():
    print("task")

ray.get(task.remote())

2021-12-05 20:05:40,272	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


 pid=6168) task


In [4]:
import ray
import logging
# Initiate a driver.

@ray.remote
class Actor:
    def __init__(self):
        # Basic config automatically configures logs to
        # be streamed to stdout and stderr.
        # Set the severity to INFO so that info logs are printed to stdout.
        logging.basicConfig(level=logging.INFO)

    def log(self, msg):
        logging.info(msg)

actor = Actor.remote()
ray.get(actor.log.remote("A log message for an actor."))

@ray.remote
def f(msg):
    logging.basicConfig(level=logging.INFO)
    logging.info(msg)

ray.get(f.remote("A log message for a task"))

 pid=6168) INFO:root:A log message for an actor.
 pid=10148) INFO:root:A log message for a task


In [5]:
import ray
# Initiate a driver.

@ray.remote
def task():
    print(f"task_id: {ray.get_runtime_context().task_id}")

ray.get(task.remote())

 pid=10148) task_id: TaskID(4ee449587774c1f0ffffffffffffffffffffffff01000000)
